#why in txt if it is array

In [1]:
import os
import numpy as np
from keras.preprocessing import image
from keras.applications import densenet
import keras
import pickle
from scipy.spatial.distance import cosine
from PIL import Image


In [2]:
model = densenet.DenseNet201(include_top=False, weights='imagenet', input_shape=(256, 256, 3), pooling='avg', classes=1000)

In [3]:
def getencodings(url):    
    _img = image.load_img(url, target_size=(256, 256))
    _img = keras.utils.img_to_array(_img)
    _img = np.expand_dims(_img, axis=0)
    _enc = densenet.preprocess_input(_img)
    _enc = model.predict(_enc)
    return _enc

In [6]:
type(getencodings("C:/Users/Aarush Raj/OneDrive/Desktop/img2rec/DEEP-CHEF-PROJECT/downloaded_images/train/0_Chicken Tikka Masala/1_Chicken Tikka Masala.jpg"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step


numpy.ndarray

In [7]:
#C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train\0_Chicken Tikka Masala\1_Chicken Tikka Masala.jpg
#C:/Users/Aarush Raj/OneDrive/Desktop/img2rec/DEEP-CHEF-PROJECT/downloaded_images/train/0_Chicken Tikka Masala/1_Chicken Tikka Masala.jpg
names_list=os.listdir(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train")
encodings_list = []
c = 0
for i in names_list[0:1]:
    p=i.split("_")[1]
    for j in range(1,9):
        image_path="C:/Users/Aarush Raj/OneDrive/Desktop/img2rec/DEEP-CHEF-PROJECT/downloaded_images/train/"+i+"/"+str(j)+"_"+p+".jpg"
        #img = image.load_img(image_path, target_size=(256, 256))
        encoding = getencodings(image_path)
        encodings_list.append(encoding)
    c += 1
    print(c)
print(len(names_list), len(encodings_list))
with open('encodings.txt', 'wb') as file:
    pickle.dump(encodings_list, file)
with open('enc_names.txt', 'wb') as file:
    pickle.dump(names_list, file)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 405ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1
358 8


In [ ]:
with open(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\aarush\encodings.txt","r+") as f:
    

In [64]:
type(encodings_list)

list

In [54]:
image_path

'C:/Users/Aarush Raj/OneDrive/Desktop/img2rec/DEEP-CHEF-PROJECT/downloaded_images/train/0_Chicken Tikka Masala/1_Chicken Tikka Masala.jpg'

just checking

In [17]:
_img = image.load_img(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\test\1_Naan\9_Naan.jpg", target_size=(256, 256))
_img = keras.utils.img_to_array(_img)
_img = np.expand_dims(_img, axis=0)
_enc = densenet.preprocess_input(_img)
_enc = model.predict(_enc)
_enc

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step


array([[9.7090713e-05, 7.5560465e-04, 1.5090167e-03, ..., 2.7459392e-01,
        1.9984721e-01, 1.2217317e-01]], dtype=float32)

In [13]:
print(os.listdir(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images"))

['hello', 'test', 'train']


just checking

In [5]:
img = image.load_img(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train\1_Naan\2_Naan.jpg", target_size=(256, 256))
img = keras.utils.img_to_array(img)
img = np.expand_dims(img, axis=0)
enc = densenet.preprocess_input(img)
enc = model.predict(enc)
print(enc)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
[[9.1914306e-05 5.8829499e-04 1.2086974e-03 ... 7.5686830e-01
  6.8234848e-03 7.4211019e-04]]


In [20]:
img1 = image.load_img(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train\1_Naan\5_Naan.jpg", target_size=(256, 256))
img1 = keras.utils.img_to_array(img1)
img1 = np.expand_dims(img1, axis=0)
enc1 = densenet.preprocess_input(img1)
enc1 = model.predict(enc1)
print(enc1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
[[1.3076351e-04 3.2538065e-04 1.4036527e-03 ... 5.8270401e-01
  1.1677598e-02 9.1031656e-02]]


In [21]:
similarity=cosine(_enc[0],enc[0])
s1=cosine(_enc[0],enc1[0])
s2=cosine(enc[0],enc1[0])
s3=cosine(enc1[0],enc[0])
print(similarity,s1,s2,s3)

0.38520339248235214 0.39427060395575675 0.38686255295183825 0.38686255295183825


In [24]:
img7 = image.load_img(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train\1_Naan\7_Naan.jpg", target_size=(256, 256))
img7 = keras.utils.img_to_array(img7)
img7 = np.expand_dims(img7, axis=0)
enc7 = densenet.preprocess_input(img7)
enc7 = model.predict(enc7)
print(enc7)

img3 = image.load_img(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train\1_Naan\3_Naan.jpg", target_size=(256, 256))
img3 = keras.utils.img_to_array(img3)
img3 = np.expand_dims(img3, axis=0)
enc3 = densenet.preprocess_input(img3)
enc3 = model.predict(enc3)
print(enc3)

print(cosine(enc3[0],enc7[0]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
[[6.5320186e-05 4.3773686e-04 1.4675669e-03 ... 1.0696392e+00
  2.8892964e-02 3.2143086e-02]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
[[1.1759455e-04 4.8694725e-04 1.9708984e-03 ... 1.1663135e+00
  9.2119738e-02 1.7724106e-02]]
0.16504474276782277


In [11]:
print(type(_enc))
print(len(_enc[0]))
print(len(enc[0]))
print(_enc[0][156])

<class 'numpy.ndarray'>
1920
1920
1.6126285e-05


In [8]:
with open("draft.txt","w") as f:
    f.write(str(enc))
    f.write(str(_enc))